# Imports


In [1]:
import os
import sys
import copy
import glob
import tqdm
from torch import nn
import random
import torch
import platform
from typing import Callable, List, Optional, Dict
import numpy as np
import scipy.sparse as sp

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoTokenizer, AutoModel

import torch_geometric
from torch_geometric.data import (
    Data,
    InMemoryDataset,
    Batch
    )
import torch_geometric.datasets as datasets
import torch_geometric.transforms as transforms
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

import umap
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, silhouette_score

# To ensure determinism
seed = 1234
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

# Check versions
print(torch.__version__)
print(torch.version.cuda)
print(platform.python_version())
print(torch_geometric.__version__)

/home/siddharthsa/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.8.1+cu101
10.1
3.8.18
1.7.0


# Filter Java Code Snippets

## Get all accepted Java submissions

In [4]:
import pandas as pd
import glob
import os

metadata_folder = "/home/siddharthsa/cs21mtech12001-Tamal/API-Misuse-Prediction/PDG-gen/Repository/Benchmarks/CodeNet/metadata"
metadata_files = glob.glob(os.path.join(metadata_folder, '*.csv'))
allowed_status = ["Accepted"]

accepted_submissions = {}
for file_loc in tqdm.tqdm(metadata_files):
    problem_id = file_loc[file_loc.rindex("/") + 1: -4]
    metadata_file = pd.read_csv(file_loc)
    accepted_submissions[problem_id] = []
    for index, row in metadata_file.iterrows():
        if row["filename_ext"] == "java" and row["status"] in allowed_status:
            code_file_location = f"/home/siddharthsa/cs21mtech12001-Tamal/API-Misuse-Prediction/PDG-gen/Repository/Benchmarks/CodeNet/data/{row['problem_id']}/Java/{row['submission_id']}.java"
            if os.path.isfile(code_file_location):
                accepted_submissions[problem_id].append(code_file_location)
            else:
                # pass
                print(code_file_location)
                print("code file doesn't exist!!")

  0%|          | 0/4053 [00:00<?, ?it/s]

100%|██████████| 4053/4053 [11:02<00:00,  6.11it/s]


## Filter the code-snippets that have only one class and one method

In [61]:
import javalang
from javalang.parser import JavaSyntaxError

def count_methods(java_file_loc):
  
  with open(java_file_loc, 'r') as f:
    java_source = f.read()

  java_class = javalang.parse.parse(java_source)
  method_count = sum(1 for _, node in java_class.filter(javalang.tree.MethodDeclaration))
  class_count = sum(1 for _, node in java_class.filter(javalang.tree.TypeDeclaration))
  return class_count, method_count


accepted_submissions_filtered = {}
for pid in tqdm.tqdm(accepted_submissions):
    accepted_submissions_filtered[pid] = []
    for path in accepted_submissions[pid]:
        try:
            #print(path)
            class_count, method_count = count_methods(path)
            if class_count == 1 and method_count == 1:
                accepted_submissions_filtered[pid].append(path) 
        except (AssertionError, JavaSyntaxError, javalang.tokenizer.LexerError):
            pass

100%|██████████| 4053/4053 [51:22<00:00,  1.31it/s]  


## Get top 5 projects by submission

In [69]:
accepted_submissions_count = {}
for pid in accepted_submissions_filtered:
    accepted_submissions_count[pid] = len(accepted_submissions_filtered[pid])
    
accepted_submissions_count_sorted = sorted(accepted_submissions_count.items(), key=lambda x:x[1], reverse = True)
print(accepted_submissions_count_sorted[:10])

sum_ = 0
for pid in accepted_submissions_count_sorted[:10]:
    sum_ += pid[1]
print("Total accepted java submissions in top 10 problems: ", sum_)

[('p02388', 2155), ('p02389', 1688), ('p02391', 1434), ('p02393', 1281), ('p02396', 1239), ('p02390', 1233), ('p02392', 1119), ('p00000', 1040), ('p02400', 993), ('p02403', 920)]
Total accepted java submissions in top 10 problems:  13102


## Copy the filtered Java files

In [70]:
import shutil

output_folder_location = "/home/siddharthsa/cs21mtech12001-Tamal/API-Misuse-Prediction/PDG-gen/Repository/Benchmarks/CodeNet/accepted_java_sols_1_method"

for pid, count in tqdm.tqdm(accepted_submissions_count_sorted):
    if count < 20:
        continue
    output_project_folder_location = output_folder_location + "/" + pid
    if not os.path.exists(output_project_folder_location):
        os.mkdir(output_project_folder_location)
    
    for original_file in accepted_submissions_filtered[pid]:
        file_name = original_file[original_file.rindex("/") + 1:]
        shutil.copyfile(original_file, output_project_folder_location + "/" + file_name)
    

100%|██████████| 4053/4053 [00:15<00:00, 265.87it/s]


# Preprocess for PDG Generation

In [6]:
import sys
import json
import pandas as pd
import random
import os
from pathlib import Path
import re
import glob
import tqdm
import pyparsing

input_folder_location = "/home/siddharthsa/cs21mtech12001-Tamal/API-Misuse-Prediction/PDG-gen/Repository/Benchmarks/CodeNet/accepted_java_sols_1_method"
output_folder_location = "/home/siddharthsa/cs21mtech12001-Tamal/API-Misuse-Prediction/PDG-gen/Repository/Benchmarks/CodeNet/processed_data_for_pdg"

projects_to_consider = ['p02388', 'p02389', 'p02391', 'p02393', 'p02396']

def copyFile(input_file_path, output_file_path):
    
    input_file = open(input_file_path, "r")
    newfile = open(output_file_path, "w+")
    
    # Remove all comments
    original_code = input_file.read()
    commentFilter = pyparsing.javaStyleComment.suppress()
    modified_code = commentFilter.transformString(original_code)

    for line in modified_code.split("\n"):
        line = line.replace('\u00A0', " ")
        
        # Skip empty lines, lines like @Test
        if len(line.strip()) == 0 or line.strip().startswith("@"):
            continue
        
        # Remove import statements and packages
        if line.startswith("import") or line.startswith("package") or line.strip().replace("\n", "").strip() == "":
            continue
        
        # Add newline at the end
        if not line.endswith("\n"):
            line += "\n"
        newfile.write(line)
    
    newfile.close()
    input_file.close()

for project in tqdm.tqdm(projects_to_consider):
    input_project_folder = input_folder_location + "/" + project
    java_files = glob.glob(os.path.join(input_project_folder, '*.java'))
    
    output_project_folder_location = output_folder_location + "/" + project
    if not os.path.exists(output_project_folder_location):
        os.mkdir(output_project_folder_location)
        
    for input_java_file_path in java_files:
        file_name = input_java_file_path[input_java_file_path.rindex("/") + 1:]
        output_java_file_path = output_project_folder_location + "/" + file_name
        copyFile(input_java_file_path, output_java_file_path)
        

100%|██████████| 5/5 [00:22<00:00,  4.51s/it]


# Get The PGDs

# Postprocess After PDG Generation